# Lab 7 - Using Redis and Arrow to store and retrieve Pandas DFs.

The actual exercises for Lab 7 are effectively just your coursework, and ask you to plan and create pipelines with X amount of stages.
The code below is technically Exercise 4.

In [5]:
import redis
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

In [6]:
redis_conn = redis.Redis(host = "127.0.0.1", port = 6379) #, db = redis_db

In [8]:
df = pd.read_csv("iris.csv", names = ["F1", "F2", "F3", "F4", "T1"])

In [11]:
table = pa.Table.from_pandas(df)
output_stream = pa.BufferOutputStream()
pq.write_table(table, output_stream)
serialized_data = output_stream.getvalue().to_pybytes()

In [12]:
redis_conn.set("iris_key", serialized_data)

True

In [14]:
retrieved_data = redis_conn.get("iris_key")
buffer_reader = pa.BufferReader(retrieved_data)
retrieved_table = pq.read_table(buffer_reader)
deserialized_df = retrieved_table.to_pandas()

In [15]:
deserialized_df.head()

,F1,F2,F3,F4,T1
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
